In [1]:
import requests
import os
import logging
from time import sleep
from urllib.parse import quote

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load API Key securely
api_key = os.getenv("TMDB_API_KEY", "ed123ac044ee7901764a4df0688d6471")
if not api_key:
    raise ValueError("API key not found. Please set the TMDB_API_KEY environment variable.")

base_url = "https://api.themoviedb.org/3"

# Language mapping
LANGUAGES = {
    "en": "English",
    "hi": "Hindi",
    "es": "Spanish",
    "fr": "French",
    "de": "German",
    "zh": "Chinese",
    "ja": "Japanese",
    "ko": "Korean",
    "it": "Italian",
    "ru": "Russian",
    "pt": "Portuguese",
    "ar": "Arabic",
    "te": "Telugu",
    # Add more as needed
}

def api_get_request(url, retries=3, timeout=10):
    """Perform an API GET request with exponential backoff."""
    delay = 1
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=timeout)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 401:
                logging.error("Invalid API Key.")
                return {}
            elif response.status_code == 429:
                logging.warning("Rate limit exceeded. Retrying...")
                sleep(delay)
                delay *= 2
            else:
                logging.error(f"Error {response.status_code}: {response.text}")
                return {}
        except requests.exceptions.RequestException as e:
            logging.error(f"Network error: {e}. Retrying...")
            sleep(delay)
            delay *= 2
    logging.error("Max retries exceeded.")
    return {}

def search_movie(movie_name, page=1):
    """Search for a movie by name."""
    encoded_name = quote(movie_name)
    url = f"{base_url}/search/movie?api_key={api_key}&query={encoded_name}&language=en-US&page={page}"
    data = api_get_request(url)
    results = data.get("results", [])
    if not results:
        print(f"No movies found for '{movie_name}'.")
    return results, data.get("total_pages", 1)

def get_streaming_providers(movie_id, region="IN"):
    """Get streaming platform details for a movie."""
    url = f"{base_url}/movie/{movie_id}/watch/providers?api_key={api_key}"
    data = api_get_request(url)
    providers = data.get("results", {}).get(region, {}).get("flatrate", [])
    if not providers:
        return "Not available for streaming."
    return ", ".join(provider["provider_name"] for provider in providers)

def get_top_cast(movie_id):
    """Fetch the top 3 cast members for a movie."""
    url = f"{base_url}/movie/{movie_id}/credits?api_key={api_key}"
    data = api_get_request(url)
    cast = data.get("cast", [])
    top_cast = [member["name"] for member in cast[:3]]
    return ", ".join(top_cast) if top_cast else "Not available"

def get_movie_details(movie_id, region="IN"):
    """Get detailed information about a movie."""
    url = f"{base_url}/movie/{movie_id}?api_key={api_key}&language=en-US"
    data = api_get_request(url)

    if not data:
        logging.error("Failed to fetch movie details.")
        return None

    original_language = data.get("original_language", "N/A")
    full_language = LANGUAGES.get(original_language, original_language.capitalize())

    return {
        "id": movie_id,
        "title": data.get("title", "N/A"),
        "genre": ", ".join(genre["name"] for genre in data.get("genres", [])) or "N/A",
        "cast": get_top_cast(movie_id),
        "year": data.get("release_date", "N/A").split("-")[0],
        "rating": data.get("vote_average", "N/A"),
        "language": full_language,
        "streaming": get_streaming_providers(movie_id, region),
        
    }

def list_genres():
    """List all available genres."""
    url = f"{base_url}/genre/movie/list?api_key={api_key}&language=en-US"
    data = api_get_request(url)
    return {genre["id"]: genre["name"] for genre in data.get("genres", [])}

def search_by_genre(genre_name, page=1):
    """Search for movies by genre."""
    genres = list_genres()
    genre_id = next((gid for gid, name in genres.items() if name.lower() == genre_name.lower()), None)
    if not genre_id:
        logging.info(f"Genre '{genre_name}' not found.")
        return [], 1

    url = f"{base_url}/discover/movie?api_key={api_key}&with_genres={genre_id}&language=en-US&page={page}"
    data = api_get_request(url)
    return data.get("results", []), data.get("total_pages", 1)

def display_movies(movies, page, total_pages):
    """Display a list of movies."""
    if not movies:
        print("No movies found.")
        return
    print(f"\nPage {page} of {total_pages}:")
    for idx, movie in enumerate(movies, 1):
        title = movie.get("title", "N/A")
        year = movie.get("release_date", "N/A").split("-")[0]
        print(f"{idx}. {title} ({year})")

def add_to_watchlist(movie, watchlist):
    """Add a movie to the watchlist."""
    if all(m['id'] != movie['id'] for m in watchlist):
        watchlist.append(movie)
        print(f"'{movie['title']}' has been added to your watchlist.")
    else:
        print("This movie is already in your watchlist.")

def safe_int_input(prompt):
    """Safely get integer input from the user."""
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Please enter a valid number.")

def user_interface():
    """Simple user interface for movie search."""
    watchlist = []
    user_region = input("Enter your region code (default is IN): ").strip().upper() or "IN"
    print("\033[1mWelcome to the Movie Search Engine!\033[0m")

    while True:
        print("\nOptions:")
        print("1. Search for a movie by name")
        print("2. Search for movies by genre")
        print("3. Add a movie to watchlist")
        print("4. View watchlist")
        print("5. Exit")
        choice = safe_int_input("Enter your choice: ")

        if choice == 1:
            movie_name = input("Enter the movie name: ")
            page = 1
            while True:
                movies, total_pages = search_movie(movie_name, page)
                display_movies(movies, page, total_pages)
                movie_choice = input("\nEnter the number of the movie for details, 'n' for next page, 'p' for previous page, or 'b' to go back: ").strip().lower()
                if movie_choice.isdigit():
                    idx = int(movie_choice) - 1
                    if 0 <= idx < len(movies):
                        details = get_movie_details(movies[idx]["id"], user_region)
                        if details:
                            print("\nMovie Details:")
                            for key, value in details.items():
                                print(f"{key.capitalize()}: {value}")
                            add_to_watchlist(details, watchlist)
                    else:
                        print("Invalid selection.")
                elif movie_choice == 'n' and page < total_pages:
                    page += 1
                elif movie_choice == 'p' and page > 1:
                    page -= 1
                elif movie_choice == 'b':
                    break
                else:
                    print("Invalid input.")

        elif choice == 2:
            genres = list_genres()
            print("\nAvailable Genres:")
            for name in genres.values():
                print(f"- {name}")
            genre_name = input("\nEnter the genre name: ")
            page = 1
            while True:
                movies, total_pages = search_by_genre(genre_name, page)
                display_movies(movies, page, total_pages)
                movie_choice = input("\nEnter the number of the movie for details, 'n' for next page, 'p' for previous page, or 'b' to go back: ").strip().lower()
                if movie_choice.isdigit():
                    idx = int(movie_choice) - 1
                    if 0 <= idx < len(movies):
                        details = get_movie_details(movies[idx]["id"], user_region)
                        if details:
                            print("\nMovie Details:")
                            for key, value in details.items():
                                print(f"{key.capitalize()}: {value}")
                            add_to_watchlist(details, watchlist)
                    else:
                        print("Invalid selection.")
                elif movie_choice == 'n' and page < total_pages:
                    page += 1
                elif movie_choice == 'p' and page > 1:
                    page -= 1
                elif movie_choice == 'b':
                    break
                else:
                    print("Invalid input.")

        elif choice == 3:
            movie_name = input("Enter the movie name to add to your watchlist: ")
            movies, _ = search_movie(movie_name)
            display_movies(movies, 1, 1)
            movie_choice = safe_int_input("\nEnter the number of the movie: ") - 1
            if 0 <= movie_choice < len(movies):
                details = get_movie_details(movies[movie_choice]["id"], user_region)
                if details:
                    add_to_watchlist(details, watchlist)

        elif choice == 4:
            if not watchlist:
                print("Your watchlist is empty.")
            else:
                print("\nYour Watchlist:")
                for idx, movie in enumerate(watchlist, 1):
                    print(f"{idx}. {movie['title']} ({movie['year']})")

        elif choice == 5:
            print("Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    user_interface()
    

Enter your region code (default is IN):  5


Welcome to the Movie Search Engine!

Options:
1. Search for a movie by name
2. Search for movies by genre
3. Add a movie to watchlist
4. View watchlist
5. Exit


Enter your choice:  5


Goodbye!
